In [3]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation,Flatten
from keras import regularizers

import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn import metrics
from keras.models import model_from_json

from tqdm import tqdm
import math

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
class Data():
    def __init__(self,data):
        self.data=data
        self.dataSmote=pd.DataFrame()
        
    def sample(self,start,end):
        size=len(self.data)
        return self.data[int(size*start):int(size*end)]
    
    def getData():
        return self.dataSmote

    def sample_smote(self,start,end):
        data=self.sample(start,end)
        self.dataSmote=np.array(data.drop("Class",axis=1))
        y=np.array(data[['Class']])
        smt=SMOTE()
        self.dataSmote,y=smt.fit_sample(self.dataSmote,y)
        self.dataSmote=pd.DataFrame(self.dataSmote)
        y=pd.DataFrame(y)
        self.dataSmote['Class']=y
        self.dataSmote=self.dataSmote.sample(frac=1)
        self.dataSmote.columns=list(data.columns)
        return self.dataSmote

In [0]:

class Aggregator():
    
    def __init__(self):
        self.wB1=0.55
        self.wB2=0.35
        self.wB3=0.10
    
    def aggregate(self,delta,B1,B2,B3):
        delta=np.array(delta)
        temp=(self.wB1*np.array(B1) + self.wB2*np.array(B2) + self.wB3*np.array(B3))
        temp-=delta
        delta+=temp

        return delta

In [0]:
class Model():
    
    def __init__(self):    # Apply differential privacy to his model training
        self.input_shape=(30,)
        self.model = Sequential()
        self.model.add(Dense(32, activation='relu',input_shape=self.input_shape,kernel_regularizer=regularizers.l2(0.01))) 
        self.model.add(Dropout(0.2))
        self.model.add(Dense(16, activation='relu'))
        self.model.add(Dropout(0.1))
        self.model.add(Dense(8, activation='relu'))
        self.model.add(Dense(1, activation='sigmoid'))
        self.model.compile(optimizer='adam',   #rmsprop
                      loss='binary_crossentropy',
                      metrics=['accuracy'])


    def run(self,X,Y,validation_split=0.0,load=True):
        self.model.fit(X,Y,epochs=1,validation_split=validation_split, verbose=1) #Validation split of 10% on the training data
        
    def evaluate(self,X,Y):
        return self.model.evaluate(X,Y)[1]*100 #Not clear what is happening here
    
    def loss(self,X,Y):
        return self.model.evaluate(X,Y)[0]
    
    def predict(self,X):
        return self.model.predict(X)

    def predictClass(self,X):
        return self.model.predict_classes(X)
        
    
    def getWeights(self):
        return self.model.get_weights()
    
    def setWeights(self,weight):
        self.model.set_weights(weight)

In [0]:
class Bank(Model):
    
    def __init__(self,data,split_size=0):
        super().__init__()
        self.data=data
        self.split(split_size)
    
    def setData(self,data,split_size=0):
        self.data=data
        self.split(split_size)
        
    def getData(self):
        return self.data
    
    def split(self,split_size):
        X=self.data.copy()
        X.drop(['Class'],axis=1,inplace=True)
        Y=self.data[['Class']]

        if split_size == 0:
            self.X_train, self.X_test, self.Y_train, self.Y_test = X,X,Y,Y
        else:
            self.X_train, self.X_test, self.Y_train, self.Y_test = train_test_split(X, Y, test_size=split_size)

In [0]:
def getWeights(weights):
    A=[]
    for i in weights:
        A.extend(np.array(i).flatten())
    B=[]
    for i in A:
        B.extend(i.flatten())

    
    return np.array(B)

def getAdversarialData(probabilities,data,gradient,result):
    output=[]
    for index,i in enumerate(probabilities.reshape(-1)):
        temp=[]
        temp.append(i)
        temp.extend(np.array(data[index:index+1])[0])
        temp.extend(gradient)
        temp.append(result)
        output.append(temp)
    return np.array(output)

In [11]:
data = pd.read_csv('/content/drive/My Drive/creditcard.csv')
data.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (29) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.5,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [12]:
data.shape

(284807, 31)

In [0]:

results={}
aggregator=Aggregator()


In [14]:
data['Amount'] = pd.to_numeric(data['Amount'],errors='coerce')
data.dtypes

Time      float64
V1        float64
V2        float64
V3        float64
V4        float64
V5        float64
V6        float64
V7        float64
V8        float64
V9        float64
V10       float64
V11       float64
V12       float64
V13       float64
V14       float64
V15       float64
V16       float64
V17       float64
V18       float64
V19       float64
V20       float64
V21       float64
V22       float64
V23       float64
V24       float64
V25       float64
V26       float64
V27       float64
V28       float64
Amount    float64
Class       int64
dtype: object

In [15]:
data= data.dropna()
data.isnull().values.any()

False

In [16]:
datum=Data(data)

Data_Test=datum.sample(0.90,1)
Data_Global=datum.sample_smote(0,0.1)              #use datum.sample  if smote not required
Data_Model_1A=datum.sample_smote(0.1,0.5)
Data_Model_2A=datum.sample_smote(0.5,0.8)
Data_Model_3A=datum.sample_smote(0.8,0.9)
#Data_Model_1B=datum.sample_smote(0.50,70)
#Data_Model_2B=datum.sample_smote(0.70,85)
#Data_Model_3B=datum.sample_smote(0.85,90)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, 

In [17]:
GlobalBank=Bank(Data_Global[0:1])
GlobalBank.run(GlobalBank.X_train, GlobalBank.Y_train,load=False)

results['BankG.1']=GlobalBank.evaluate(GlobalBank.X_test,GlobalBank.Y_test)

Epoch 1/1
1/1 [==============================] - 0s 31ms/step


In [0]:
step=1
no_iter=500 # for speed


In [19]:
result=[]
for i in tqdm(range(1,501)):

    Bank1=Bank(Data_Model_1A[i*step:(i+1)*step]) #extracting first 1 point of the dataset
    Bank1.run(Bank1.X_train,Bank1.Y_train) #Training on the first data point
    Bank1_gradient=getWeights(GlobalBank.getWeights()) - getWeights(Bank1.getWeights()) #Calculating the gradient
    result.extend(
        getAdversarialData(Bank1.predict(Bank1.X_test),Bank1.X_test,Bank1_gradient,True)
    ) #adding it to the new database
#1 point added to the dataset

    Bank2=Bank(Data_Model_2A[i*step:(i+1)*step]) 
    Bank2.run(Bank2.X_train,Bank2.Y_train)
    Bank2_gradient=getWeights(GlobalBank.getWeights()) - getWeights(Bank2.getWeights())
    result.extend(
        getAdversarialData(Bank2.predict(Bank2.X_test),Bank2.X_test,Bank2_gradient,True)
    )
#1 point added to the dataset


    Bank3=Bank(Data_Model_3A[i*step:(i+1)*step])
    Bank3.run(Bank3.X_train,Bank3.Y_train)
    Bank3_gradient=getWeights(GlobalBank.getWeights()) - getWeights(Bank3.getWeights())
    result.extend(
        getAdversarialData(Bank3.predict(Bank3.X_test),Bank3.X_test,Bank3_gradient,True)
    )
#1 point added to the dataset



    result.extend(
        getAdversarialData(Bank1.predict(Bank2.X_test),Bank2.X_test,Bank1_gradient,False)
    )
#1 point added to the dataset

    result.extend(
        getAdversarialData(Bank1.predict(Bank3.X_test),Bank3.X_test,Bank1_gradient,False)
    )
#1 point added to the dataset

    result.extend(
        getAdversarialData(Bank2.predict(Bank1.X_test),Bank1.X_test,Bank2_gradient,False)
    )
#1 point added to the dataset

    result.extend(
        getAdversarialData(Bank2.predict(Bank3.X_test),Bank3.X_test,Bank2_gradient,False)
    )
#1 point added to the dataset
    result.extend(
        getAdversarialData(Bank3.predict(Bank1.X_test),Bank1.X_test,Bank3_gradient,False)
    )
#1 point added to the dataset
    result.extend(
        getAdversarialData(Bank3.predict(Bank2.X_test),Bank2.X_test,Bank3_gradient,False)
    )

#1 point added to the dataset
    delta=aggregator.aggregate(GlobalBank.getWeights(),Bank1.getWeights(),Bank2.getWeights(),Bank3.getWeights())

    GlobalBank.setWeights(delta)


    if(i%1 == 0 and i!=0):
        result=np.array(result)
        advData=pd.DataFrame(result)

        loc='Adversarial_Input'
        loc+=str(i)
        loc+='.csv'

        advData.to_csv(loc,index=False)
        result=[]

  0%|          | 0/500 [00:00<?, ?it/s]

Epoch 1/1
1/1 [==============================] - 0s 159ms/step - loss: 0.3146 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 156ms/step - loss: 0.3039 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 151ms/step - loss: 0.3209 - accuracy: 1.0000


  0%|          | 1/500 [00:02<23:24,  2.81s/it]

Epoch 1/1
1/1 [==============================] - 0s 178ms/step - loss: 13679.5645 - accuracy: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 149ms/step - loss: 0.3190 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 153ms/step - loss: 9345.5742 - accuracy: 0.0000e+00


  0%|          | 2/500 [00:05<23:02,  2.78s/it]

...

100%|█████████▉| 498/500 [54:46<00:26, 13.09s/it]

Epoch 1/1
1/1 [==============================] - 0s 222ms/step - loss: 36540.6758 - accuracy: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 229ms/step - loss: 0.3138 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 225ms/step - loss: 0.3256 - accuracy: 1.0000


100%|█████████▉| 499/500 [54:58<00:12, 12.84s/it]

Epoch 1/1
1/1 [==============================] - 0s 233ms/step - loss: 0.3034 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 241ms/step - loss: 2526.3501 - accuracy: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 246ms/step - loss: 0.3222 - accuracy: 1.0000


100%|██████████| 500/500 [55:11<00:00,  6.62s/it]


In [1]:
loc

NameError: ignored

In [20]:
print(results)

{'BankG.1': 50.44474005699158}


In [20]:
advData

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682,1683,1684,1685,1686,1687,1688,1689,1690,1691,1692,1693,1694,1695,1696
0,0.0,41396.000000,1.159803,0.011840,1.025612,1.350963,-0.809957,-0.302297,-0.380594,0.004566,0.742218,-0.181267,-0.779963,0.574586,0.265352,-0.330410,0.025106,-0.109917,-0.201975,-0.034888,-0.238193,-0.110639,0.096410,0.518234,-0.147978,0.436138,0.658705,-0.217487,0.060747,0.031695,14.950000,0.237057,-0.086915,0.109903,0.113055,-0.201762,0.412838,-0.092077,-0.178714,0.341041,...,-0.082625,-0.395279,0.161219,-0.033983,0.221757,-0.307772,0.434569,-0.060222,0.002083,0.363151,0.473154,0.524461,-0.128769,-0.004124,0.064948,0.096946,-0.693933,0.219989,0.544843,0.013323,-0.348253,0.501134,-0.000083,0.000000,0.0,0.000000,-0.000026,0.000076,-0.000024,7.460669e-07,0.290523,0.853379,0.167941,0.032147,0.291259,-0.771327,-0.317156,0.225181,0.000124,1.0
1,0.0,86108.589443,-2.202582,1.819371,-2.635981,3.161165,-3.672247,0.843824,-2.919379,-1.212333,-1.602360,-4.041358,1.518586,-2.515835,0.110103,-3.560226,-0.505770,-3.236424,-4.776713,-1.491930,1.901469,0.606258,-0.650505,0.907054,0.262488,0.026927,0.363889,-0.042775,0.524711,-0.023520,316.756383,0.469923,0.208194,-0.035470,-0.260386,0.216073,0.198577,-0.179042,0.027528,0.269137,...,0.847587,-0.559428,-0.255730,-0.040241,0.022280,-0.094904,0.130603,0.148572,-0.267674,-0.354126,0.451759,-0.065418,-0.302510,0.192393,0.694749,0.485422,0.088762,-0.347733,0.146671,0.002102,0.524095,0.441501,0.000744,-0.000744,0.0,-0.000744,0.000000,-0.000744,0.000000,7.441299e-04,0.429779,-0.225237,0.752006,-0.001278,-0.020195,-0.343990,-1.266562,0.763683,-0.000744,1.0
2,0.0,148478.706572,-1.494717,3.816631,-7.537627,3.206531,-1.244079,-2.052800,-2.398819,1.456949,-2.559775,-5.087569,1.907890,-6.015462,-1.188286,-6.670775,-0.058201,-4.349565,-5.627283,-1.624890,0.997610,0.121880,1.096064,1.063505,0.065289,0.264553,-0.679129,-0.309686,0.571037,0.387674,119.427756,0.467248,-0.168585,0.051876,-0.186388,-0.264893,-0.002285,0.117814,0.072428,-0.145990,...,0.618739,0.077590,-0.262679,0.159369,-0.478824,0.250922,-0.444954,0.243391,0.249435,-0.352219,0.319610,0.006751,0.278516,0.474002,0.154713,0.205281,0.008209,0.430229,0.159003,0.020940,-0.301852,0.035393,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,-0.641080,0.290573,1.142170,-0.280926,0.842527,-0.436392,-0.079015,-0.299734,0.000000,1.0
3,0.0,86108.589443,-2.202582,1.819371,-2.635981,3.161165,-3.672247,0.843824,-2.919379,-1.212333,-1.602360,-4.041358,1.518586,-2.515835,0.110103,-3.560226,-0.505770,-3.236424,-4.776713,-1.491930,1.901469,0.606258,-0.650505,0.907054,0.262488,0.026927,0.363889,-0.042775,0.524711,-0.023520,316.756383,0.237057,-0.086915,0.109903,0.113055,-0.201762,0.412838,-0.092077,-0.178714,0.341041,...,-0.082625,-0.395279,0.161219,-0.033983,0.221757,-0.307772,0.434569,-0.060222,0.002083,0.363151,0.473154,0.524461,-0.128769,-0.004124,0.064948,0.096946,-0.693933,0.219989,0.544843,0.013323,-0.348253,0.501134,-0.000083,0.000000,0.0,0.000000,-0.000026,0.000076,-0.000024,7.460669e-07,0.290523,0.853379,0.167941,0.032147,0.291259,-0.771327,-0.317156,0.225181,0.000124,0.0
4,0.0,148478.706572,-1.494717,3.816631,-7.537627,3.206531,-1.244079,-2.052800,-2.398819,1.456949,-2.559775,-5.087569,1.907890,-6.015462,-1.188286,-6.670775,-0.058201,-4.349565,-5.627283,-1.624890,0.997610,0.121880,1.096064,1.063505,0.065289,0.264553,-0.679129,-0.309686,0.571037,0.387674,119.427756,0.237057,-0.086915,0.109903,0.113055,-0.201762,0.412838,-0.092077,-0.178714,0.341041,...,-0.082625,-0.395279,0.161219,-0.033983,0.221757,-0.307772,0.434569,-0.060222,0.002083,0.363151,0.473154,0.524461,-0.128769,-0.004124,0.064948,0.096946,-0.693933,0.219989,0.544843,0.013323,-0.348253,0.501134,-0.000083,0.000000,0.0,0.000000,-0.000026,0.000076,-0.000024,7.460669e-07

In [20]:
loc

'Adversarial_Input500.csv'

In [0]:
advData=pd.DataFrame()
for i in range(1,501,1):
    loc='Adversarial_Input'
    loc+=str(i)
    loc+='.csv'
    x=pd.read_csv(loc)
    advData=advData.append(x,ignore_index=True)

In [22]:
advData.shape

(4500, 1697)

In [23]:
advData.head


<bound method NDFrame.head of         0              1         2  ...      1694      1695  1696
0     0.0   72914.000000 -0.569007  ... -0.625554  0.001000   1.0
1     0.0  111448.000000 -1.173111  ... -0.920786  0.001000   1.0
2     1.0  150309.067279 -2.593686  ...  0.001362  0.001000   1.0
3     0.0  111448.000000 -1.173111  ... -0.625554  0.001000   0.0
4     0.0  150309.067279 -2.593686  ... -0.625554  0.001000   0.0
...   ...            ...       ...  ...       ...       ...   ...
4495  1.0  152759.000000  1.860938  ...  0.486801 -0.000409   0.0
4496  0.0   50772.997291 -8.926849  ... -0.118850 -0.001409   0.0
4497  0.0  152759.000000  1.860938  ... -0.118850 -0.001409   0.0
4498  1.0   50772.997291 -8.926849  ... -0.879055 -0.000409   0.0
4499  1.0  109493.357503  1.040881  ... -0.879055 -0.000409   0.0

[4500 rows x 1697 columns]>

In [0]:
X = pd.read_csv('Adversarial_Input200.csv')

In [36]:
X.head

<bound method NDFrame.head of      0         1         2         3  ...      1693      1694      1695  1696
0  1.0   76239.0  1.525596 -1.056489  ...  0.251553  0.293775  0.000744   1.0
1  0.0  138688.0  2.140622 -0.956778  ...  0.499732 -0.012454  0.000000   1.0
2  0.0  152718.0  1.927380 -0.616018  ...  0.389064  0.038693  0.000000   1.0
3  1.0  138688.0  2.140622 -0.956778  ...  0.251553  0.293775  0.000744   0.0
4  1.0  152718.0  1.927380 -0.616018  ...  0.251553  0.293775  0.000744   0.0
5  0.0   76239.0  1.525596 -1.056489  ...  0.499732 -0.012454  0.000000   0.0
6  0.0  152718.0  1.927380 -0.616018  ...  0.499732 -0.012454  0.000000   0.0
7  0.0   76239.0  1.525596 -1.056489  ...  0.389064  0.038693  0.000000   0.0
8  0.0  138688.0  2.140622 -0.956778  ...  0.389064  0.038693  0.000000   0.0

[9 rows x 1697 columns]>

In [0]:
from keras.layers import Conv2D, MaxPooling2D
def Model(input_shape):

    model = Sequential()

    model.add(Conv2D(64, 2,input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32,2))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())

    model.add(Dense(32, activation='relu',kernel_regularizer=regularizers.l2(0.01))) 
    model.add(Dropout(0.2))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam',   #rmsprop
                  loss= "binary_crossentropy",#[custom_loss()]
                  metrics=['acc']
                 )

    return model

In [0]:
from imblearn.under_sampling import RandomUnderSampler

In [0]:
X=advData.copy()
X.drop(['1696'],axis=1,inplace=True)
Y=advData[['1696']]

In [28]:
undersample = RandomUnderSampler(sampling_strategy='majority')
undersample = RandomUnderSampler(sampling_strategy=1)
X,Y = undersample.fit_resample(X,Y)
input_shape=(X.shape[0],X.shape[1],)
X=np.array(X)
X = X.reshape(X.shape[0],212,8,1)
input_shape=X.shape[1:]
Y=np.array(Y)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [29]:
model=Model(input_shape)
model.fit(X,Y,epochs=10,validation_split=0.1, verbose=1)

Train on 2700 samples, validate on 300 samples
Epoch 1/10
2700/2700 [==============================] - 5s 2ms/step - loss: 13.5742 - acc: 0.4981 - val_loss: 0.8616 - val_acc: 0.0533
Epoch 2/10
2700/2700 [==============================] - 4s 2ms/step - loss: 0.7583 - acc: 0.5500 - val_loss: 0.7302 - val_acc: 0.0000e+00
Epoch 3/10
2700/2700 [==============================] - 4s 2ms/step - loss: 0.6963 - acc: 0.5556 - val_loss: 0.7460 - val_acc: 0.0000e+00
Epoch 4/10
2700/2700 [==============================] - 4s 2ms/step - loss: 0.6941 - acc: 0.5556 - val_loss: 0.7626 - val_acc: 0.0000e+00
Epoch 5/10
2700/2700 [==============================] - 4s 2ms/step - loss: 0.6930 - acc: 0.5556 - val_loss: 0.7746 - val_acc: 0.0000e+00
Epoch 6/10
2700/2700 [==============================] - 4s 2ms/step - loss: 0.6924 - acc: 0.5556 - val_loss: 0.7859 - val_acc: 0.0000e+00
Epoch 7/10
2700/2700 [==============================] - 4s 2ms/step - loss: 0.6919 - acc: 0.5556 - val_loss: 0.7932 - val_acc: 0